# RAG & Tamplate baseline code
## input
1. input_file_path
2. prompt - human_message_prompt
## 수정할 내용
1. system_message_prompt

In [5]:
import os
import bs4

import openai
import faiss 
from dotenv import load_dotenv
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # Now this import should work

from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

load_dotenv()


openai.api_key = ''

In [7]:
from langchain.document_loaders import TextLoader

# 파일 경로 설정
input_file_path = 'KakaoTalk_20240815_1527_10_362_현동.txt'

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader(input_file_path)
datas = loader.load()

# # 로드된 문서를 출력합니다.
# for data in datas:
#     print(data)
datas

[Document(metadata={'source': 'KakaoTalk_20240815_1527_10_362_현동.txt'}, page_content='근이 님과 카카오톡 대화\n저장한 날짜 : 2024-08-15 15:27:22\n\n--------------- 2024년 7월 15일 월요일 ---------------\n[김현동] [오전 10:34] 뭐하느데\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] 는데\n[김현동] [오전 10:34] ㅁㅎ는데\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] ㅁㅎ\n[근이] [오전 10:34] ㄱㅁ\n[근이] [오전 10:34] 시험\n[근이] [오전 10:34] 2시간전이다\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 농구\n[김현동] [오전 10:34] 뭐야\n[근이] [오전 10:34] 어캐\n[김현동] [오전 10:34] 자신있나\n[근이] [오전 10:34] 돼ㅑㅆ는데\n[김현동] [오전 10:34] 자신 있냐고\n[근이] [오전 10:34] 당연하지\n[근이] [오전 10:34] 자시간\n[근이] [오전 10:34] 2024프로\n[김현동] [오전 10:34] 이번엔 합격해야지\n[근이] [오전 10:34] 합격해야지\n[근이] [오전 10:34] 농구\n[김현동] [오전 10:34] 농구 바로 탈락\n[김현동] [오전 10:34] 바로 집 가기\n[근이] [오전 10:34] 2승\n[근이] [오전 10:34] 했따매\n[김현동] [오전 10:34] 본선에서\n[근이] [오전 10:34] 또\n[김현동] [오전 10:34] 바로 탈락\n[근이] [오전 10:35] 몇골\n[김현동] [오전 10:35] 0\n[근이] [오전 10:35] 거짓말\n[근이] [오전 10:35] 

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(datas)
splits

[Document(metadata={'source': 'KakaoTalk_20240815_1527_10_362_현동.txt'}, page_content='근이 님과 카카오톡 대화\n저장한 날짜 : 2024-08-15 15:27:22'),
 Document(metadata={'source': 'KakaoTalk_20240815_1527_10_362_현동.txt'}, page_content='--------------- 2024년 7월 15일 월요일 ---------------\n[김현동] [오전 10:34] 뭐하느데\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] 는데\n[김현동] [오전 10:34] ㅁㅎ는데\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] ㅁㅎ\n[김현동] [오전 10:34] ㅁㅎ\n[근이] [오전 10:34] ㄱㅁ\n[근이] [오전 10:34] 시험\n[근이] [오전 10:34] 2시간전이다\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 왜\n[근이] [오전 10:34] 농구\n[김현동] [오전 10:34] 뭐야\n[근이] [오전 10:34] 어캐\n[김현동] [오전 10:34] 자신있나\n[근이] [오전 10:34] 돼ㅑㅆ는데\n[김현동] [오전 10:34] 자신 있냐고\n[근이] [오전 10:34] 당연하지\n[근이] [오전 10:34] 자시간\n[근이] [오전 10:34] 2024프로\n[김현동] [오전 10:34] 이번엔 합격해야지\n[근이] [오전 10:34] 합격해야지\n[근이] [오전 10:34] 농구\n[김현동] [오전 10:34] 농구 바로 탈락\n[김현동] [오전 10:34] 바로 집 가기\n[근이] [오전 10:34] 2승\n[근이] [오전 10:34] 했따매\n[김현동] [오전 10:34] 본선에서\n[근이] [오전 10:34] 또\n[김현동] [오전 10:34

In [9]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai.api_key))

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
print(vectorstore)
print(retriever)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A17FEA5A10>


In [10]:
prompt = hub.pull("rlm/rag-prompt",api_key=openai.api_key)
prompt

c:\Users\SKT041\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [11]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0,api_key=openai.api_key)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
"""
# 짜야할 promt template
# 키보드
1. 대체어 제시 (input 입력 중인 단어 / 문장)
# 레포트 
1. 부모용 레포트 (사이버 폭력 / 언어습관)
2. 자녀용 레포트 (언어 습관)
"""


string_template = '''
해당 대화에서 {person}(열정)의 언어습관을 분석해줘.
'''

system_message_prompt = SystemMessagePromptTemplate.from_template(string_template) # 고정 입력
human_message_prompt = HumanMessagePromptTemplate.from_template('{person}') # 바뀌는 입력 
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt]) # 1,2 합쳐서 GPT에 입력

prompt = chat_prompt.format_prompt(person='김현동').to_messages()
print(prompt[0].content)


해당 대화에서 김현동(열정)의 언어습관을 분석해줘.



In [28]:
result = rag_chain.invoke(
prompt[0].content
)  #문서에 대한 질의를 입력하고, 답변을 출력합니다.

In [29]:
result

'김현동의 언어습관은 간결하고 직설적이며, 자주 줄임말을 사용합니다. 또한, 친구를 격려하거나 응원할 때도 짧고 강한 표현을 사용합니다. 대화 중 반복적으로 "좋은데"와 같은 긍정적인 피드백을 주는 경향이 있습니다.'